In [1]:
import psycopg2
import sqlalchemy as sq
import pandas as pd
import numpy as np
import mariadb
import dask.dataframe as dd
import json
import os
import shutil
import subprocess
from pathlib import Path
import pyodbc

In [2]:
conn_lims = pyodbc.connect("Driver={SQL Server};"
                            "Server=192.168.5.18\CROPNUT;"
                            "Database=cropnuts;"
                            "uid=thomasTsuma;pwd=GR^KX$uRe9#JwLc6")

<>:2: SyntaxWarning: invalid escape sequence '\C'
<>:2: SyntaxWarning: invalid escape sequence '\C'
C:\Users\tommy\AppData\Local\Temp\ipykernel_7480\232176420.py:2: SyntaxWarning: invalid escape sequence '\C'
  "Server=192.168.5.18\CROPNUT;"


In [3]:
soil_df = pd.read_sql("select sample_code, batch_date, crop_name, chemical_name, result, unit_name from SampleResults where group_name='Soil Analysis'",con=conn_lims)

C:\Users\tommy\AppData\Local\Temp\ipykernel_7480\166756027.py:1: UserWarning: pandas only supports SQLAlchemy connectable (engine/connection) or database string URI or sqlite3 DBAPI2 connection. Other DBAPI2 objects are not tested. Please consider using SQLAlchemy.
  soil_df = pd.read_sql("select sample_code, batch_date, crop_name, chemical_name, result, unit_name from SampleResults where group_name='Soil Analysis'",con=conn_lims)


In [4]:
soil_df.to_csv("input/soil_samples.csv")

In [8]:
soil_df = pd.read_csv("input/soil_samples.csv")

In [9]:
soil_df

,Unnamed: 0,sample_code,batch_date,crop_name,chemical_name,result,unit_name
0,0,CF045SA0784,2012-07-09 00:00:00.000,Tomatoes,Iron,165,ppm
1,1,CF045SA0784,2012-07-09 00:00:00.000,Tomatoes,C.E.C,16.6,meq/100g
2,2,CF045SA0784,2012-07-09 00:00:00.000,Tomatoes,Aluminium,481,ppm
3,3,CF045SA0784,2012-07-09 00:00:00.000,Tomatoes,EC (Salts),61.0,uS/cm
4,4,CF045SA0784,2012-07-09 00:00:00.000,Tomatoes,Organic Matter,2.36,%
...,...,...,...,...,...,...,...
3938686,3938686,CK626SA0164,2024-04-30 00:00:00.000,No crop,Exchangeable Aluminium,0.67,meq/100g
3938687,3938687,CK626SA0164,2024-04-30 00:00:00.000,No crop,Total Nitrogen,0.097,%
3938688,3938688,CK626SA0164,2024-04-30 00:00:00.000,No crop,Total Carbon,1.31,%
3938689,3938689,CK626SA0165,2024-04-30 00:00:00.000,No crop,pH,6.00,NaN


In [10]:
soil_df = soil_df.dropna(subset='result')

In [32]:
soil_df = soil_df.loc[~(soil_df['result'] == "ND") ]
soil_df = soil_df.loc[~(soil_df['result'] == "Detected") ]
soil_df = soil_df.loc[~(soil_df['result'] == "Clay") ]
soil_df = soil_df.loc[~(soil_df['result'] == "HClay") ]
soil_df = soil_df.loc[~(soil_df['result'] == "CLoam") ]
soil_df = soil_df.loc[~(soil_df['result'] == "Loam") ]
soil_df = soil_df.loc[~(soil_df['result'] == "SCLoam") ]
soil_df = soil_df.loc[~(soil_df['result'] == "SLoam") ]
soil_df = soil_df.loc[~(soil_df['result'] == "LSand") ]
soil_df = soil_df.loc[~(soil_df['result'] == "SClay") ]

In [33]:
soil_df['result'] = [ float(i.replace("<","").replace(">","").strip()) for i in soil_df['result']]

In [34]:
soil_df

,Unnamed: 0,sample_code,batch_date,crop_name,chemical_name,result,unit_name
0,0,CF045SA0784,2012-07-09 00:00:00.000,Tomatoes,Iron,165.000,ppm
1,1,CF045SA0784,2012-07-09 00:00:00.000,Tomatoes,C.E.C,16.600,meq/100g
2,2,CF045SA0784,2012-07-09 00:00:00.000,Tomatoes,Aluminium,481.000,ppm
3,3,CF045SA0784,2012-07-09 00:00:00.000,Tomatoes,EC (Salts),61.000,uS/cm
4,4,CF045SA0784,2012-07-09 00:00:00.000,Tomatoes,Organic Matter,2.360,%
...,...,...,...,...,...,...,...
3938686,3938686,CK626SA0164,2024-04-30 00:00:00.000,No crop,Exchangeable Aluminium,0.670,meq/100g
3938687,3938687,CK626SA0164,2024-04-30 00:00:00.000,No crop,Total Nitrogen,0.097,%
3938688,3938688,CK626SA0164,2024-04-30 00:00:00.000,No crop,Total Carbon,1.310,%
3938689,3938689,CK626SA0165,2024-04-30 00:00:00.000,No crop,pH,6.000,NaN


In [35]:
soil_df['unit_name'] = [ str(i).strip() for i in soil_df['unit_name'].values ]

In [36]:
soil_df['chemical_name'] = [ str(i).lower().replace(" ","_").replace("(","").replace(")","").replace(".","").strip() for i in soil_df['chemical_name'].values ]

In [37]:
soil_df = soil_df.loc[~(soil_df['result'] < 0)]

In [38]:
soil_df = soil_df.loc[~(soil_df['unit_name'] == "nan")]

In [40]:
soil_df.to_csv("input/soil_analysis_cleaned.csv")